In [4]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

load_dotenv(find_dotenv(), override=True)

llm = ChatOpenAI()

model_names = {
	"gpt-3.5": "gpt-3.5-turbo-0125",
	"gpt-4": "gpt-4-turbo",
	"opus": "claude-3-opus-20240229",
	"sonnet": "claude-3-sonnet-2024022",
	"haiku": "claude-3-haiku-20240307"}


In [5]:
def ask_gpt(question, model=model_names.get("gpt-3.5")):
	return llm.invoke(question, model=model)


def ask_gpt_3(question):
	return ask_gpt(question).content


def ask_gpt_4(question):
	return ask_gpt(question, model=model_names.get("gpt-4")).content

# Anthropic Model

In [12]:
def ask_claude(question, model):
	chat_model = ChatAnthropic(model=model)
	return chat_model.invoke(question).content


def ask_opus(question):
	return ask_claude(question, model_names.get("opus"))


def ask_sonnet(question):
	return ask_claude(question, model_names.get("sonnet"))


def ask_haiku(question):
	return ask_claude(question, model_names.get("haiku"))

In [ ]:
from langchain.schema import (
	SystemMessage,
	AIMessage,
	HumanMessage
)

messages = [
	SystemMessage(content="You are an expert in writing dutch poems"),
	HumanMessage(content="write a short poem in dutch and end with the name of your Creator. OpenAi or Anthropic")
]

# Caching LLM Responses

## in memory Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

In [ ]:
%%time
# set_llm_cache(InMemoryCache())
# ask_gpt_3("What is your funniest joke?")

In [ ]:
%%time
ask_opus("Who made you?")

In [ ]:
%%time 
ask_gpt_4("who are you? what model are you based on? Chatgpt 3 or chatgpt 4")

## SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# first request not in cache takes longer
ask_gpt_3("Tell me a joke")

# second (cached, faster)
ask_gpt_3("Tell me a joke")

## LLM Streaming

In [ ]:
llm = ChatOpenAI()
for chunk in llm.stream("What gpt model are you? 3 or 4 and write a poem about that", model="gpt-4-turbo"):
	print(chunk.content, end="", flush=True)



# PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an expert in building a RAG machine learning model and running it locally with Langchain and HuggingFace models. The models used will be {model} and the programming language used will be {language}. We will run it on the OS {OS}.'''

prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(model="Llama2", language="Python", OS="MacOS")

prompt

In [ ]:
ask_opus(prompt)

In [ ]:
ask_gpt_4(prompt)

# ChatPromptTemplates 

In [1]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
	[
		SystemMessage(content='You respond only in JSON format.'),
		HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population')
	]
)

messages = chat_template.format_messages(n='10', area='Europe')

print(messages)

[SystemMessage(content='You respond only in JSON format.'), HumanMessage(content='Top 10 countries in Europe by population')]


In [8]:
print(ask_gpt_3(messages))

{
    "1": {
        "country": "Germany",
        "population": 83783942
    },
    "2": {
        "country": "France",
        "population": 65273511
    },
    "3": {
        "country": "United Kingdom",
        "population": 67886011
    },
    "4": {
        "country": "Italy",
        "population": 60461826
    },
    "5": {
        "country": "Spain",
        "population": 46754778
    },
    "6": {
        "country": "Ukraine",
        "population": 43733762
    },
    "7": {
        "country": "Poland",
        "population": 37887768
    },
    "8": {
        "country": "Romania",
        "population": 19237691
    },
    "9": {
        "country": "Netherlands",
        "population": 17134872
    },
    "10": {
        "country": "Belgium",
        "population": 11589623
    }
}


In [10]:
print(ask_haiku(messages))

Here are the top 10 countries in Europe by population:

{
  "data": [
    {
      "rank": 1,
      "country": "Russia",
      "population": 146,748,590
    },
    {
      "rank": 2,
      "country": "Germany",
      "population": 83,166,711
    },
    {
      "rank": 3,
      "country": "United Kingdom",
      "population": 67,886,011
    },
    {
      "rank": 4,
      "country": "France",
      "population": 67,391,582
    },
    {
      "rank": 5,
      "country": "Italy",
      "population": 60,367,477
    },
    {
      "rank": 6,
      "country": "Spain",
      "population": 46,754,778
    },
    {
      "rank": 7,
      "country": "Poland",
      "population": 37,846,611
    },
    {
      "rank": 8,
      "country": "Romania",
      "population": 19,237,691
    },
    {
      "rank": 9,
      "country": "Netherlands",
      "population": 17,134,872
    },
    {
      "rank": 10,
      "country": "Belgium",
      "population": 11,589,623
    }
  ]
}


In [13]:
print(ask_opus(messages))

Here are the top 10 countries in Europe by population, in JSON format:

[
  {
    "rank": 1,
    "country": "Russia",
    "population": 145934462
  },
  {
    "rank": 2,
    "country": "Germany",
    "population": 83783942
  },
  {
    "rank": 3,
    "country": "United Kingdom",
    "population": 67886011
  },
  {
    "rank": 4,
    "country": "France",
    "population": 65273511
  },
  {
    "rank": 5,
    "country": "Italy",
    "population": 60461826
  },
  {
    "rank": 6,
    "country": "Spain",
    "population": 46754778
  },
  {
    "rank": 7,
    "country": "Ukraine",
    "population": 43733762
  },
  {
    "rank": 8,
    "country": "Poland",
    "population": 37846611
  },
  {
    "rank": 9,
    "country": "Romania",
    "population": 19237691
  },
  {
    "rank": 10,
    "country": "Netherlands",
    "population": 17134872
  }
]


# Simple Chains

In [15]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model=model_names.get("gpt-3.5"))

template = '''You are an expert in planning everyday tasks. I want you to order the following tasks: {tasks} by the following criteria: {criteria}'''

prompt_template = PromptTemplate.from_template(template=template)

# prompt = prompt_template.format(tasks="cleaning the house, studying for school, buying a gift for my wife",
#                                 criteria="importance")

chain = LLMChain(
	llm=llm,
	prompt=prompt_template,
	verbose=True
)

output = chain.invoke(
	{'tasks': 'studying for school, buying a gift for my wife"s birthday which is today, cleaning the house',
	 'criteria': 'importance'}
)

In [16]:
print(output)

{'tasks': 'studying for school, buying a gift for my wife"s birthday which is today, cleaning the house', 'criteria': 'importance', 'text': "1. Buying a gift for your wife's birthday (since it is today and time-sensitive)\n2. Studying for school (to ensure you stay on top of your academic responsibilities)\n3. Cleaning the house (while important, can potentially be done at a later time if necessary)"}


In [17]:
tasks = input('Enter tasks: ')
criteria = input('Enter criteria: ')


In [19]:
output=chain.invoke({'tasks': tasks, 'criteria': criteria})

print(output['text'])

1. Sleep
2. Cleaning the house


# Sequential chains

Make a series of calls to one or more LLM's. You can take the output from one chain and use it as the input to another chain.

## 2 types:
1. SimpleSequentialChain
2. General form of sequential chains

### SimpleSequentialChain
- Series of chains, each individual chain has a single input and a single output. 
- Output is used as input to the next step

### General Form of sequential chains


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Experienced python programmer
gpt_3_5 = ChatOpenAI(model=model_names.get('gpt-3.5'), temperature=1.2)
prompt_template_programmer = 'You are an expert in Python programming. Write a function that implements the concept of {concept}'
prompt_programmer = PromptTemplate.from_template(
	template=prompt_template_programmer
)

opus = ChatAnthropic(model=model_names.get('opus'))

